## Virtual Environment for Individual-Based Modeling

This virtual environment is based on an Agent-Based Modeling system using some assumptions that are derived from previousobservations of waterbirds’ behaviors within some habitats in the tropics.

Created on March 31, 2019 by Ralph Florent <r.florent@jacobs-university.de>

In [2]:
# Virtual Environment for Individual-Based Modeling (IBM)
#
# Created on March 31, 2019
#
# Authors:
#   Ralph Florent <r.florent@jacobs-university.de>
#   Davi Tavares <davi.tavares@leibniz-zmt.de>
#   Agostino Merico <a.merico@jacobs-university.de>
#
# Main entry point for the application

# ==============================================================================
# START: Preamble
# ==============================================================================

# -*- coding: utf-8 -*-

import config
import constants as CONST
from core import initialize, observe, update
from helpers import make_gif

# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
# reload external python modules
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# ==============================================================================
# END: Preamble
# ==============================================================================

# ==============================================================================
# START: Application
# ==============================================================================

# main entry point for the application
# TODO: proper docs
def application():
    
    # initialize internal config for the app
    config.init()
    
    # define stopwatch for the process
    time = 0
    
    habitats, agents = initialize()
    observe(habitats, agents, time)
    
    # loop over t-1 time
    print('=> START: Processing random movements based on prob dist')
    
    for time in range(1, CONST.PROCESSING_TIME):
        updated_habitats, updated_agents = update(agents)
        observe(updated_habitats, updated_agents, time)
    
    print('=> END: Processing random movements based on prob dist')
    make_gif(CONST.MAIN_DIRECTORY, 'image.gif', CONST.IMAGE_STORAGE)

    
# run application
application()

# ==============================================================================
# END: Application
# ==============================================================================

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
=> START: Processing random movements based on prob dist
-- habitat: one
---- short: s:0.40040, w:0.01136, d:0.00155, f:0.53659
---- overall prob: 0.0000037844
-- habitat: one
---- short: s:0.40040, w:0.01136, d:0.00155, f:0.53659
---- overall prob: 0.0000037844
-- habitat: two
---- long: s:0.00340, w:0.00840, d:0.00619, f:0.52179
---- overall prob: 0.0000000922
-- habitat: one
---- short: s:0.40040, w:0.01136, d:0.00128, f:0.49972
---- overall prob: 0.0000029077
-- habitat: one
---- short: s:0.40040, w:0.01136, d:0.00128, f:0.49972
---- overall prob: 0.0000029077
-- habitat: one
---- long: s:0.00620, w:0.00869, d:0.00632, f:0.50102
---- overall prob: 0.0000001706
-- habitat: two
---- long: s:0.00340, w:0.00840, d:0.00619, f:0.52179
---- overall prob: 0.0000000922
-- habitat: two
---- long: s:0.00340, w:0.00840, d:0.00619, f:0.52179
---- overall prob: 0.0000000922
-- habitat: one
---- short: s:0.400